In [2]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated
import operator
import os


In [7]:
load_dotenv()

True

In [9]:
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")


In [10]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [11]:
@tool
def calculator(expression: str) -> str:
    """
    Calculate mathematical expressions.
    
    Args:
        expression: Math expression like "2 + 2" or "15 * 37"
    """
    try:
        result = eval(expression, {"_builtins_": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"
@tool
def search(query: str) -> str:
    """
    Search for information (simulated).
    
    Args:
        query: The search query
    """
    # Simulated search results
    knowledge = {
        "python": "Python is a high-level programming language created in 1991.",
        "langgraph": "LangGraph is a framework for building stateful multi-actor applications.",
        "react": "ReAct is an agent pattern that combines reasoning and acting."
    }  
    for key, value in knowledge.items(): 

_IncompleteInputError: incomplete input (966054253.py, line 28)

In [12]:
@tool
def calculator(expression: str) -> str:
    """
    Calculate mathematical expressions.
    
    Args:
        expression: Math expression like "2 + 2" or "15 * 37"
    """
    try:
        result = eval(expression, {"_builtins_": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"
@tool
def search(query: str) -> str:
    """
    Search for information (simulated).
    
    Args:
        query: The search query
    """
    # Simulated search results
    knowledge = {
        "python": "Python is a high-level programming language created in 1991.",
        "langgraph": "LangGraph is a framework for building stateful multi-actor applications.",
        "react": "ReAct is an agent pattern that combines reasoning and acting."
    }  
    for key, value in knowledge.items():
        if key in query.lower():
            return value
    return "no information found."
tools = [calculator,search]
print("✅ Tools created")


✅ Tools created


In [ ]:
# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

# system prompt
react_prompt = SystemMessage(content="""You are a helpful assistant with tools.

Use calculator for math and search for information.
Think step-by-step before using tools.""")

# Define nodes
def react_assistant(state: MessagesState) -> dict:
    """Agent node - reasons and decides which tool to use."""
    messages = [react_prompt]+ state["messages"]
    response = llm_with_tools.invoke(messages)
    return{"messages": [response]}

def should_continue(state: mess)